## Neural Networks classification with Pytorch

For readers interested in a more detailed exploration of PyTorch, we recommend referring to the official PyTorch website and its extensive documentation and tutorials (https://pytorch.org/tutorials/).

In [2]:
!pip install torch

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/58/b8/51b956c2da9729390a3080397cd2f31171394543af7746681466e372f69a/torch-2.2.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/b7/f4/6a90020cd2d93349b442bfcb657d0dc91eee65491600b2cb1d388bc98e6b/typing_extensions-4.9.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/198.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/198.6 MB 991.0 kB/s eta 0:03:21
   ---------------------------------------- 0.1/198.6 MB 787.7 kB/s eta 0:04:13
   ---------------------------------------- 0.2/198.6 MB 1.3 MB/s eta 0:02:32
   ---------------------------------------- 0.3/198.6 MB 1.8 MB/s eta 0:01:51
   ---------------------------------------- 0.5/198.6 MB 2.3 MB/s eta 0:01:27
   ---------------------------------------- 0.8/198.6 MB 3.0 MB/s eta 0:01:06
   ------------------------

In [3]:
# We first need to import torch
# If you dont have torch, visit https://pytorch.org/get-started/locally/ and go through the setup steps

# Import torch and check version
import torch
print(torch.__version__)

2.2.0+cpu


In [4]:
# We can check if cuda is available on our device and enable it
# Otherwise we will work on cpu
if torch.cuda.is_available():
    device = "cuda" # Use NVIDIA GPU (if available)
elif torch.backends.mps.is_available():
    device = "mps" # Use Apple Silicon GPU (if available)
else:
    device = "cpu" # Default to CPU if no GPU is available

In [5]:
device

'cpu'

In [6]:
import pandas as pd

# Load the dataset for the binary classification
df = pd.read_csv('binary_data.csv')


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split into features and labels
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values


In [8]:
X.shape, y.shape

((64626, 82), (64626,))

In [9]:
from sklearn.preprocessing import LabelEncoder

#'y' is our categorical labels array
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(y)
encoded_labels

array([0, 0, 0, ..., 0, 0, 0])

In [10]:
# Split the data 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    encoded_labels, 
                                                    test_size=0.2, 
                                                    random_state=42) 
len(X_train), len(X_test), len(y_train), len(y_test)

(51700, 12926, 51700, 12926)

In [11]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler on the training data and transform both training and testing data
X_train= scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [12]:
# Turning training the data into tensors
X_train = torch.from_numpy(X_train).type(torch.float)
y_train = torch.from_numpy(y_train).type(torch.long)

In [13]:
# Turning test data into tensors
X_test = torch.from_numpy(X_test).type(torch.float)
y_test = torch.from_numpy(y_test).type(torch.long)

In [14]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([51700, 82]),
 torch.Size([51700]),
 torch.Size([12926, 82]),
 torch.Size([12926]))

In [15]:
import torch.nn as nn

# Define a  model class that inherits from nn.Module
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()

        # First linear layer with 82 inputs and 64 outputs
        self.fc1 = nn.Linear(82, 64)

        # Additional linear layers to increase model complexity
        self.fc2 = nn.Linear(64, 32)  # Second layer 
        self.fc3 = nn.Linear(32, 64)  # Third layer 

        # Final output layer for binary classification
        # Outputs a single value from 64 input features
        self.fc4 = nn.Linear(64, 1)

        # Activation function to introduce non-linearity
        self.relu = nn.ReLU()

    # Define the forward pass through the network
    def forward(self, x):
        # Apply layers with ReLU activation functions in between for non-linearity
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))

        # No activation function in the final layer
        x = self.fc4(x)
        return x

# 'device' is defined (e.g., as 'cuda' or 'cpu')
# Create an instance of the model and send it to the specified device
binary_model = BinaryClassifier().to(device)
binary_model

BinaryClassifier(
  (fc1): Linear(in_features=82, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
)

In [16]:
# loss and optimizer function
criterion = torch.nn.BCEWithLogitsLoss() # The Sigmoid() activation function is build-in
optimizer = torch.optim.Adam(params=binary_model.parameters(), lr=0.01)

In [17]:

# Assuming binary_model, criterion, optimizer are defined

torch.manual_seed(42)
epochs = 5000

# Put all data on target device
X_train, y_train = X_train.to(device), y_train.to(device).float()
X_test, y_test = X_test.to(device), y_test.to(device).float()

for epoch in range(epochs):
    # Training Phase
    binary_model.train()
    y_logits = binary_model(X_train).squeeze()
    y_pred = torch.round(torch.sigmoid(y_logits))

    # Calculate loss
    loss_value = criterion(y_logits, y_train)

    # Calculate accuracy
    correct_train = (y_pred == y_train).sum().item()
    train_accuracy = 100 * correct_train / len(y_train)

    # Zero gradients, backward pass, optimize
    optimizer.zero_grad()
    loss_value.backward()
    optimizer.step()

    # Testing Phase
    binary_model.eval()
    with torch.no_grad():
        test_logits = binary_model(X_test).squeeze()
        test_pred = torch.round(torch.sigmoid(test_logits))
        test_loss = criterion(test_logits, y_test)

        correct_test = (test_pred == y_test).sum().item()
        test_accuracy = 100 * correct_test / len(y_test)

    if epoch % 100 == 0:
        print(f"Epoch: {epoch} | Train Loss: {loss_value.item()}, Train Accuracy: {train_accuracy:.3f}% | Test Loss: {test_loss.item()}, Test Accuracy: {test_accuracy:.3f}%")

binary_model.train()


Epoch: 0 | Train Loss: 0.7342821955680847, Train Accuracy: 35.424% | Test Loss: 0.6737154126167297, Test Accuracy: 56.429%
Epoch: 100 | Train Loss: 7.392031693598256e-05, Train Accuracy: 100.000% | Test Loss: 0.0006136756855994463, Test Accuracy: 99.992%
Epoch: 200 | Train Loss: 1.3196521649661008e-05, Train Accuracy: 100.000% | Test Loss: 0.00036186532815918326, Test Accuracy: 99.992%
Epoch: 300 | Train Loss: 6.089216185500845e-06, Train Accuracy: 100.000% | Test Loss: 0.0002779583155643195, Test Accuracy: 99.992%
Epoch: 400 | Train Loss: 3.561901849025162e-06, Train Accuracy: 100.000% | Test Loss: 0.0002372047893004492, Test Accuracy: 99.992%
Epoch: 500 | Train Loss: 2.3489174054702744e-06, Train Accuracy: 100.000% | Test Loss: 0.0002092629874823615, Test Accuracy: 99.992%
Epoch: 600 | Train Loss: 1.6435882344012498e-06, Train Accuracy: 100.000% | Test Loss: 0.00019003415945917368, Test Accuracy: 99.992%
Epoch: 700 | Train Loss: 1.235895638274087e-06, Train Accuracy: 100.000% | Test 

BinaryClassifier(
  (fc1): Linear(in_features=82, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
)

In [18]:
import numpy as np

# Evaluating the model
binary_model.eval()

with torch.no_grad():
    y_preds = torch.round(torch.sigmoid(binary_model(X_test))).squeeze()

# Ensure y_preds is of the same data type and device as y_test_encoded
y_preds = y_preds.type_as(y_test)

# Calculate the number of correct predictions
correct_predictions = (y_preds == y_test).sum()

# Calculate the accuracy percentage
accuracy_percentage = 100 * correct_predictions.item() / len(y_test)

print(f"Accuracy: {accuracy_percentage:.2f}%")


Accuracy: 100.00%
